In [13]:
import os
import re
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse import hstack, csr_matrix


In [14]:

df = pd.read_csv("../../csv_files/reddit_controversal_df_features.csv")

TEXT_COL = "text_clean"
LABEL_COL = "controversial_flag"
# Set aside numeric features
NUM_COLS = [
    "comment_upvote_ratio", "sentiment"   
]

use_cols = [TEXT_COL, LABEL_COL] + NUM_COLS
df_model = df[use_cols].dropna().copy()

In [15]:
X_train_df, X_test_df = train_test_split(
    df_model,
    test_size=0.2,
    random_state=42,
    stratify=df_model[LABEL_COL]
)

y_train = X_train_df[LABEL_COL].astype(int).values
y_test  = X_test_df[LABEL_COL].astype(int).values


In [16]:
tfidf = TfidfVectorizer(
    max_features=8000,
    ngram_range=(1, 2),
    min_df=2
)

X_train_text = tfidf.fit_transform(X_train_df[TEXT_COL])
X_test_text  = tfidf.transform(X_test_df[TEXT_COL])


In [17]:
X_train_num = csr_matrix(X_train_df[NUM_COLS].astype(float).values)
X_test_num  = csr_matrix(X_test_df[NUM_COLS].astype(float).values)


In [18]:
X_train = hstack([X_train_text, X_train_num])
X_test  = hstack([X_test_text, X_test_num])


In [19]:
rf = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced"  
)

rf.fit(X_train, y_train)


,n_estimators,300
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [20]:
pred = rf.predict(X_test)

print("Confusion matrix:\n", confusion_matrix(y_test, pred))
print("\nReport:\n", classification_report(y_test, pred, digits=3))


Confusion matrix:
 [[882 255]
 [295   9]]

Report:
               precision    recall  f1-score   support

           0      0.749     0.776     0.762      1137
           1      0.034     0.030     0.032       304

    accuracy                          0.618      1441
   macro avg      0.392     0.403     0.397      1441
weighted avg      0.598     0.618     0.608      1441



In [21]:
# Directly address recall: there was a bad bias to the munority class.
# 295 false negatives and 9 true negatives
# Class 0 reasonably well separated, but almost never predicting class 1
# Biased towards the majority class
# Changing decision threshold, and stronger class weighting

In [22]:
probs = rf.predict_proba(X_test)[:, 1]

for t in [0.5, 0.4, 0.3, 0.25]:
    pred = (probs >= t).astype(int)
    print(f"\nThreshold = {t}")
    print(confusion_matrix(y_test, pred))
    print(classification_report(y_test, pred, digits=3))



Threshold = 0.5
[[882 255]
 [295   9]]
              precision    recall  f1-score   support

           0      0.749     0.776     0.762      1137
           1      0.034     0.030     0.032       304

    accuracy                          0.618      1441
   macro avg      0.392     0.403     0.397      1441
weighted avg      0.598     0.618     0.608      1441


Threshold = 0.4
[[821 316]
 [280  24]]
              precision    recall  f1-score   support

           0      0.746     0.722     0.734      1137
           1      0.071     0.079     0.075       304

    accuracy                          0.586      1441
   macro avg      0.408     0.401     0.404      1441
weighted avg      0.603     0.586     0.595      1441


Threshold = 0.3
[[778 359]
 [247  57]]
              precision    recall  f1-score   support

           0      0.759     0.684     0.720      1137
           1      0.137     0.188     0.158       304

    accuracy                          0.579      1441
   macro

In [23]:
# We can see here that the model does not have an easy way of distinguishing what is controversial. 
# It can only detect controversial comments weakly
# THe issue isn't the threshold, the model is underfitting due to insufficient features.
# High bias caused by weak feature representations

# Going back into data cleaning to add more sentiment analysis (Vader) and counting conflict words

In [25]:
df = pd.read_csv("../../csv_files/reddit_controversal_df_features.csv")

In [26]:
TEXT_COL = "text_clean"
LABEL_COL = "controversial_flag"
# Set aside numeric features
NUM_COLS = [
    "comment_upvote_ratio", "sentiment", "vader_neg", "vader_neu", "vader_pos", "vader_compound",
    "conflict_count", "has_conflict",
    "exclamations", "questions", "all_caps_ratio", "comment_count", "vote_total"  
]

use_cols = [TEXT_COL, LABEL_COL] + NUM_COLS
df_model = df[use_cols].dropna().copy()

In [27]:
X_train_df, X_test_df = train_test_split(
    df_model,
    test_size=0.2,
    random_state=42,
    stratify=df_model[LABEL_COL]
)

y_train = X_train_df[LABEL_COL].astype(int).values
y_test  = X_test_df[LABEL_COL].astype(int).values


In [28]:


tfidf = TfidfVectorizer(
    max_features=8000,
    ngram_range=(1, 2),
    min_df=2
)

X_train_text = tfidf.fit_transform(X_train_df[TEXT_COL])
X_test_text  = tfidf.transform(X_test_df[TEXT_COL])


In [29]:
X_train_num = csr_matrix(X_train_df[NUM_COLS].astype(float).values)
X_test_num  = csr_matrix(X_test_df[NUM_COLS].astype(float).values)

In [30]:
X_train = hstack([X_train_text, X_train_num])
X_test  = hstack([X_test_text, X_test_num])


In [31]:
rf = RandomForestClassifier(
    n_estimators=400,
    max_depth=None,
    min_samples_leaf=5,
    class_weight={0: 1, 1: 5},
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)


,n_estimators,400
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,5
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [32]:
pred = rf.predict(X_test)

print("Confusion matrix:\n", confusion_matrix(y_test, pred))
print("\nReport:\n", classification_report(y_test, pred, digits=3))


Confusion matrix:
 [[753 384]
 [227  77]]

Report:
               precision    recall  f1-score   support

           0      0.768     0.662     0.711      1137
           1      0.167     0.253     0.201       304

    accuracy                          0.576      1441
   macro avg      0.468     0.458     0.456      1441
weighted avg      0.642     0.576     0.604      1441



In [33]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

for t in [0.5, 0.4, 0.3, 0.2, 0.1]:
    preds = (probs >= t).astype(int)
    print(f"\nTHRESHOLD = {t}")
    print(confusion_matrix(y_test, preds))
    print(classification_report(y_test, preds, digits=3))



THRESHOLD = 0.5
[[882 255]
 [295   9]]
              precision    recall  f1-score   support

           0      0.749     0.776     0.762      1137
           1      0.034     0.030     0.032       304

    accuracy                          0.618      1441
   macro avg      0.392     0.403     0.397      1441
weighted avg      0.598     0.618     0.608      1441


THRESHOLD = 0.4
[[821 316]
 [280  24]]
              precision    recall  f1-score   support

           0      0.746     0.722     0.734      1137
           1      0.071     0.079     0.075       304

    accuracy                          0.586      1441
   macro avg      0.408     0.401     0.404      1441
weighted avg      0.603     0.586     0.595      1441


THRESHOLD = 0.3
[[778 359]
 [247  57]]
              precision    recall  f1-score   support

           0      0.759     0.684     0.720      1137
           1      0.137     0.188     0.158       304

    accuracy                          0.579      1441
   macro

In [34]:
# Added more features - polarity extremeness, disagreement features, engagement imbalance

In [40]:
df = pd.read_csv("../../csv_files/reddit_controversal_df_features.csv")
TEXT_COL = "text_clean"
LABEL_COL = "controversial_flag"
# Set aside numeric features
NUM_COLS = [
    "hour", "day_of_week", "is_weekend",
    "comment_upvote_ratio", "sentiment", "vader_neg", "vader_neu", "vader_pos", "vader_compound",
    "conflict_count", "has_conflict", "abs_sentiment", "abs_vader_compound", "disagree_count", "has_disagree",
    "exclamations", "questions", "all_caps_ratio", "comment_count", "vote_total",  
    "post_length",
    "first_person_count", "second_person_count",
    "first_person_ratio", "second_person_ratio"
]

use_cols = [TEXT_COL, LABEL_COL] + NUM_COLS
df_model = df[use_cols].dropna().copy()

In [41]:
X_train_df, X_test_df = train_test_split(
    df_model,
    test_size=0.2,
    random_state=42,
    stratify=df_model[LABEL_COL]
)

y_train = X_train_df[LABEL_COL].astype(int).values
y_test  = X_test_df[LABEL_COL].astype(int).values

tfidf = TfidfVectorizer(
    max_features=8000,
    ngram_range=(1, 2),
    min_df=2
)

X_train_text = tfidf.fit_transform(X_train_df[TEXT_COL])
X_test_text  = tfidf.transform(X_test_df[TEXT_COL])


In [42]:
X_train_num = csr_matrix(X_train_df[NUM_COLS].astype(float).values)
X_test_num  = csr_matrix(X_test_df[NUM_COLS].astype(float).values)

In [43]:
X_train = hstack([X_train_text, X_train_num])
X_test  = hstack([X_test_text, X_test_num])

In [44]:
rf = RandomForestClassifier(
    n_estimators=400,
    max_depth=None,
    min_samples_leaf=5,
    class_weight={0: 1, 1: 5},
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

,n_estimators,400
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,5
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [45]:
pred = rf.predict(X_test)

print("Confusion matrix:\n", confusion_matrix(y_test, pred))
print("\nReport:\n", classification_report(y_test, pred, digits=3))

Confusion matrix:
 [[728 409]
 [222  82]]

Report:
               precision    recall  f1-score   support

           0      0.766     0.640     0.698      1137
           1      0.167     0.270     0.206       304

    accuracy                          0.562      1441
   macro avg      0.467     0.455     0.452      1441
weighted avg      0.640     0.562     0.594      1441



In [46]:
for t in [0.5, 0.4, 0.3, 0.2, 0.1]:
    preds = (probs >= t).astype(int)
    print(f"\nTHRESHOLD = {t}")
    print(confusion_matrix(y_test, preds))
    print(classification_report(y_test, preds, digits=3))



THRESHOLD = 0.5
[[882 255]
 [295   9]]
              precision    recall  f1-score   support

           0      0.749     0.776     0.762      1137
           1      0.034     0.030     0.032       304

    accuracy                          0.618      1441
   macro avg      0.392     0.403     0.397      1441
weighted avg      0.598     0.618     0.608      1441


THRESHOLD = 0.4
[[821 316]
 [280  24]]
              precision    recall  f1-score   support

           0      0.746     0.722     0.734      1137
           1      0.071     0.079     0.075       304

    accuracy                          0.586      1441
   macro avg      0.408     0.401     0.404      1441
weighted avg      0.603     0.586     0.595      1441


THRESHOLD = 0.3
[[778 359]
 [247  57]]
              precision    recall  f1-score   support

           0      0.759     0.684     0.720      1137
           1      0.137     0.188     0.158       304

    accuracy                          0.579      1441
   macro

In [ ]:
# Despite extensive feature engineering and threshold tuning, the model demonstrates that controversial comments are difficult to identify from textual and sentiment features alone. Recall can be increased substantially by lowering the decision threshold, but this comes at the cost of precision, indicating that controversy is not a strongly separable class under content-only representations.